In [1]:
from config import *
import pandas as pd
import numpy as np
from main_package.utils import data_path_to_abs_path
from main_package.bkt_pyKT import train_bkt, evaluate_bkt

In [2]:
train_data = "piech/train_original.csv"
test_data = "piech/test_original.csv"

def read_sessions(filename):
    f = open(data_path_to_abs_path(filename))
    sessions = []
    while True:
        line = f.readline()
        if line == "":
            break
        num_records = int(line.strip())
        skills = np.array(f.readline().strip().split(",")[:-1], dtype=int)
        answers = np.array(f.readline().strip().split(",")[:-1], dtype=int)
        if num_records > 1:
            sessions.append((skills, answers))
    f.close()
    return sessions

sessions_train = read_sessions(train_data)
sessions_test = read_sessions(test_data)

In [3]:
sessions_test

[(array([51, 51, 51, 51]), array([0, 1, 1, 1])),
 (array([82, 82, 82, 82, 82, 82, 82, 82, 82]),
  array([0, 0, 1, 1, 1, 1, 0, 0, 1])),
 (array([ 0,  0,  0,  0,  0,  9,  9,  9, 11, 11, 12, 12, 12, 12, 12, 14, 14,
         14, 14, 14]),
  array([1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])),
 (array([ 37,  37,  45,  45,  45,  45,  55,  55,  55,  55,  55,  55,  79,
          79,  79,  79,  79,  79,  79,  80,  80,  80,  80,  80,  82,  82,
          82,  82,  82,  82,  82, 123, 123, 123, 123, 123, 123, 123, 123,
         123, 123, 123, 123, 123, 123, 123, 123, 123, 123]),
  array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 1, 0])),
 (array([  6,   6,   6,   6,   6,   6,   6,   8,   8,   8,   8,   8,  10,
          10,  14,  14,  14,  14,  14,  14,  14,  14,  14,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          3

In [3]:
def sessions_to_df(sessions):
    df = pd.DataFrame(sessions, columns=['concepts', 'responses'])
    df['is_repeat'] = df['responses'].apply(np.zeros_like)
    return df

In [4]:
df_test = sessions_to_df(sessions_test)
df_train = sessions_to_df(sessions_train)
#df_train = df_train[:100]

In [6]:
df_test.head()

,concepts,responses,is_repeat
0,"[51, 51, 51, 51]","[0, 1, 1, 1]","[0, 0, 0, 0]"
1,"[82, 82, 82, 82, 82, 82, 82, 82, 82]","[0, 0, 1, 1, 1, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0, 9, 9, 9, 11, 11, 12, 12, 12, 1...","[1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[37, 37, 45, 45, 45, 45, 55, 55, 55, 55, 55, 5...","[0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[6, 6, 6, 6, 6, 6, 6, 8, 8, 8, 8, 8, 10, 10, 1...","[0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
bkt_trained_params = train_bkt(df_train)

training time was: 5m 33s
optimized by: L-BFGS-B
p_L0 = 0.7652508681782891
p_G = 0.07298805337969722
p_S = 0.1
p_T = 0.10185873222189407
p_F = 0.019692929412561285
      fun: 93.44605205733991
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2.57216469e-04,  1.08713039e-03, -2.07803765e+01,  6.57962574e-04,
        1.13686838e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 114
      nit: 14
     njev: 19
   status: 0
  success: True
        x: array([0.76525087, 0.07298805, 0.1       , 0.10185873, 0.01969293])


In [8]:
bkt_trained_params

[0.7652508681782891,
 0.07298805337969722,
 0.1,
 0.10185873222189407,
 0.019692929412561285]

In [5]:
bkt_trained_params = [0.76525, 0.072988, 0.1, 0.1018587, 0.0196929]

In [8]:
bkt_forget_trained_params_piech = [0.60927831, 0.18646139, 0.1, 0.1242378, 0.02519369]
bkt_standard_trained_params = [0.60379278, 0.3, 0.09999995, 0.09401142, 0] # for this I get auc 0.7286668 as expected

In [7]:
evaluate_bkt(bkt_trained_params, df_train)

number of predicted questions: 407880
auc: 0.7991332023288147
accuracy: 0.7578871236638227
